# The answer is in section "QUESTION 3"

#### Web scraping from Wikipedia

In [1]:
# import the library we use to open URLs
import urllib.request

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [4]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [5]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [6]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")

In [7]:
# the table we are searching for is the 'wikitable sortable'. Let's find it
right_table=soup.find('table', class_='wikitable sortable')
#right_table

#### Create the Dataframe

Let's start looping through the rows to get the data for every borough in the table.

There are three columns in our table that we want to scrape the data from to store our data in.

Let's use the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. Then let's set up a FOR loop for each row within that array and set Python to loop through the rows, one by one.

Within the loop I'm going to use use find_all again to search each row for <td> tags with the ‘td’ string. I'll add all of these to a variable called ‘cells’ and then check to make sure that there are 3 items in our ‘cells’ array.

If there are then we use the find(text=True)) option to extract the content string from within each <td> element in that row and add them to the A-C lists we created at the start of this step. Let’s have a look at the code:

In [8]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [9]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

### Cleaning the Dataframe

In [10]:
# Ignore cells with a borough that is Not assigned.
new_df = df[~df.Borough.str.contains("Not assigned") == True]

In [11]:
new_df = new_df.reset_index(drop=True)

### Second question

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


#### Let's download the csv file with our lat and long data

In [13]:
csv_path = 'http://cocl.us/Geospatial_data'

tor_df = pd.read_csv(csv_path)

### Let's merge new_df (with Borough and Neighborhood) with tor_df (with Lat and Long)

In [14]:
result = pd.concat([new_df, tor_df], axis = 1, sort=False)

#this line of code will drop duplicate column -to avoid the NaN problem-
result = result.loc[:,~result.columns.duplicated()]

# QUESTION 3

In [15]:
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

## Let's create a df with all the borough that contain the word Toronto

In [17]:
downtown_data = result[result['Borough'] == 'Downtown Toronto\n'].reset_index(drop=True)
west_data = result[result['Borough'] == 'West Toronto\n'].reset_index(drop=True)
east_data = result[result['Borough'] == 'East Toronto\n'].reset_index(drop=True)
central_data = result[result['Borough'] == 'Central Toronto\n'].reset_index(drop=True)

toronto_data = pd.concat([downtown_data, west_data, east_data, central_data])
toronto_data = toronto_data.reset_index(drop=True)

toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
6,M6G,Downtown Toronto,Christie,43.753259,-79.329656
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Isla...",43.695344,-79.318389
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572


In [18]:
#Foursquare credential

CLIENT_ID = 'MAZ3ZS4MDA4ONSCIAJC1VT5FQXOITDDE2FL5SFLD0MYCN2AG' # your Foursquare ID
CLIENT_SECRET = 'A5N0GPLMWF3OE2PVQ5SROJQ4N5RUAZMWJN1PKWSMIE3R5ZW2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

### Let's define the same function used in the lab to get all the nearby venues

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Let's get all the nearby venues to our toronto_data dataframe

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Regent Park, Harbourfront

Queen's Park, Ontario Provincial Government

Garden District, Ryerson

St. James Town

Berczy Park

Central Bay Street

Christie

Richmond, Adelaide, King

Harbourfront East, Union Station, Toronto Islands

Toronto Dominion Centre, Design Exchange

Commerce Court, Victoria Hotel

University of Toronto, Harbord

Kensington Market, Chinatown, Grange Park

CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport

Rosedale

Stn A PO Boxes

St. James Town, Cabbagetown

First Canadian Place, Underground city

Church and Wellesley

Dufferin, Dovercourt Village

Little Portugal, Trinity

Brockton, Parkdale Village, Exhibition Place

High Park, The Junction South

Parkdale, Roncesvalles

Runnymede, Swansea

The Beaches

The Danforth West, Riverdale

India Bazaar, The Beaches West

Studio District

Business reply mail Processing Centre, South Central Letter Processing Plant Toronto

Lawrence Park

Roselawn

Davisville N

In [21]:
print(toronto_venues.shape)

(482, 7)


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,4,4,4,4,4,4
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
"Commerce Court, Victoria Hotel",4,4,4,4,4,4
Davisville,4,4,4,4,4,4


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 164 uniques categories.


### Let's start to analyze each Neighborhood and then we'll procede with the clustering

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# define a list of column names
cols = toronto_onehot.columns.tolist()
cols

# move the column name to the beggining
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#then use .reindex() function to reorder
toronto_onehot = toronto_onehot.reindex(columns= cols)

#check result
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Belgian Restaurant,Board Shop,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,Nightclub,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Summer Camp,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
toronto_onehot.shape

(482, 164)

#### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
to_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
to_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Belgian Restaurant,Board Shop,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,Nightclub,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Summer Camp,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.066667,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.133333,0.0,0.0,0.033333,0.0,0.0,0.0,0.10,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.033333,0.00,0.0,0.0,0.0,0.0,0.0,0.033333,0.033333,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.033333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [27]:
to_grouped.shape

(39, 164)

#### Let's get each neighborhood along with the top 5 most common venues and put them into a pandas dataframe

In [28]:
num_top_venues = 5

for hood in to_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

----Berczy Park
----
----Brockton, Parkdale Village, Exhibition Place
----
----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
----
----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
----
----Central Bay Street
----
----Christie
----
----Church and Wellesley
----
----Commerce Court, Victoria Hotel
----
----Davisville
----
----Davisville North
----
----Dufferin, Dovercourt Village
----
----First Canadian Place, Underground city
----
----Forest Hill North & West, Forest Hill Road Park
----
----Garden District, Ryerson
----
----Harbourfront East, Union Station, Toronto Islands
----
----High Park, The Junction South
----
----India Bazaar, The Beaches West
----
----Kensington Market, Chinatown, Grange Park
----
----Lawrence Park
----
----Little Portugal, Trinity
----
----Moore Park, Summerhill East
----
----North Toronto West,  Lawrence Park
----
----Parkdale, Roncesvalles
----
----Queen's Park,

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's take a look at our dataframe

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = to_grouped['Neighborhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Yoga Studio,Brewery,Seafood Restaurant,Sandwich Place,Cheese Shop,Park,Comfort Food Restaurant
2,"Business reply mail Processing Centre, South C...",Pizza Place,Sandwich Place,Mobile Phone Shop,Bus Line,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice
3,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Garden,Auto Workshop,Brewery,Spa,Burrito Place,Restaurant,Farmers Market,Fast Food Restaurant,Butcher
4,Central Bay Street,Pizza Place,Bank,Pharmacy,Coffee Shop,College Stadium,Curling Ice,Donut Shop,Dog Run,Discount Store,Diner


### Let's proceed with the Clustering.
I tried different k, but everytime the bigger cluster was the one with a park and a cafe place in the first positions.

In [31]:
# set number of clusters
kclusters = 5

to_grouped_clustering = to_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

to_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Intersection,Breakfast Spot,Electronics Store,Mexican Restaurant,Rental Car Location,Bank,Medical Center,Curling Ice,Dog Run,Discount Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,0,Gas Station,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Coworking Space,Creperie
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Garden,Creperie,Discount Store,Garden Center,Diner,Dessert Shop
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Gym Pool,Pizza Place,Cosmetics Shop,Sandwich Place,Breakfast Spot,Bank
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,3,Cafeteria,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Curling Ice


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's verify my assumption

In [37]:
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Intersection,Breakfast Spot,Electronics Store,Mexican Restaurant,Rental Car Location,Bank,Medical Center,Curling Ice,Dog Run,Discount Store
1,Downtown Toronto,0,Gas Station,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Coworking Space,Creperie
2,Downtown Toronto,0,General Entertainment,Skating Rink,Café,College Stadium,Garden,Creperie,Discount Store,Garden Center,Diner,Dessert Shop
3,Downtown Toronto,0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Gym Pool,Pizza Place,Cosmetics Shop,Sandwich Place,Breakfast Spot,Bank
5,Downtown Toronto,0,Pizza Place,Bank,Pharmacy,Coffee Shop,College Stadium,Curling Ice,Donut Shop,Dog Run,Discount Store,Diner
6,Downtown Toronto,0,Park,Food & Drink Shop,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
7,Downtown Toronto,0,Airport,Park,Snack Place,Construction & Landscaping,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
8,Downtown Toronto,0,Park,Bus Stop,Pharmacy,Beer Store,Video Store,Skating Rink,Cosmetics Shop,Curling Ice,Discount Store,Diner
9,Downtown Toronto,0,Park,Fast Food Restaurant,Gym,Brewery,Ice Cream Shop,Sandwich Place,Movie Theater,Board Shop,Restaurant,Italian Restaurant
10,Downtown Toronto,0,Park,Playground,Summer Camp,Restaurant,College Stadium,Comfort Food Restaurant,Discount Store,Diner,Dessert Shop,College Rec Center


In [35]:
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,1,Baseball Field,Yoga Studio,Curling Ice,Eastern European Restaurant,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega


In [36]:
to_merged.loc[to_merged['Cluster Labels'] == 4, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,4,Construction & Landscaping,Baseball Field,Yoga Studio,Curling Ice,Eastern European Restaurant,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop
